# Format .ALI

### In order to run Modeller script, you need to change the kernel to not use the virtual env once the Modeller is installed on your machine, click on the kernel on top right of this cell and chose a python kernel

### Import libraries and loading dependencies

In [7]:
import pandas as pd  # For data manipulation and analysis
from modeller import *  # For working with Modeller tools
from modeller.automodel import *  # For automatic model building using Modeller
from pandarallel import pandarallel  # For parallel processing of pandas DataFrames
import os  # For file and directory operations

pandarallel.initialize() # Initializing pandarallel for parallel processing


INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


### Define global variables

In [8]:
# Define global variables and Modeller environment
env = Environ()  # Create a Modeller environment object

# File paths for input and output data
CSV_PATH = "../../data/csv/"  # Path to CSV files
PDB_PATH = '../../data/pdb/blast/'  # Path to PDB files
PIR_PATH = '../../data/pir/'  # Path to PIR files
ALI_PATH = '../../data/ali/'  # Path to ALI files

### Generating .ALI files

In [9]:
# Loading the variant data from a CSV file
variant_df = pd.read_csv(f'{CSV_PATH}fasta_variant.csv', sep=';')  # Load data into a pandas DataFrame
variant_df.head()  # Display the first few rows of the DataFrame for verification


,gene,identifier,variant,fasta,swiss_model,phyre2,colab_alphafold2,i_tasser,modeller,rosetta,alphafold3
0,atpE,Rv1305,atpE_p.Ala63Pro,MDPTIAAGALIGGGLIMAGGAIGAGIGDGVAGNALISGVARQPEAQ...,concluded,concluded,concluded,not_concluded,not_concluded,not_concluded,concluded
1,atpE,Rv1305,atpE_p.Asp28Ala,MDPTIAAGALIGGGLIMAGGAIGAGIGAGVAGNALISGVARQPEAQ...,concluded,concluded,concluded,not_concluded,not_concluded,not_concluded,concluded
2,atpE,Rv1305,atpE_p.Asp28Gly,MDPTIAAGALIGGGLIMAGGAIGAGIGGGVAGNALISGVARQPEAQ...,concluded,concluded,concluded,not_concluded,not_concluded,not_concluded,concluded
3,atpE,Rv1305,atpE_p.Asp28Val,MDPTIAAGALIGGGLIMAGGAIGAGIGVGVAGNALISGVARQPEAQ...,concluded,concluded,concluded,not_concluded,not_concluded,not_concluded,concluded
4,atpE,Rv1305,atpE_p.Glu61Asp,MDPTIAAGALIGGGLIMAGGAIGAGIGDGVAGNALISGVARQPEAQ...,concluded,concluded,concluded,not_concluded,not_concluded,not_concluded,concluded


In [10]:
# Function to process each variant and generate .ali files
def process_variant(row):
    gene = row['gene']  # Extract the gene name from the current row
    variant_raw = row['variant']  # Extract the raw variant name from the current row
    variant = row['variant'].replace("_p.", "_")  # Replace "_p." in the variant name with "_"
    ali_file_path = os.path.join(ALI_PATH, f"{variant}.ali")  # Construct the .ali file path

    # Check if the .ali file already exists
    if not os.path.exists(ali_file_path):
        aln = Alignment(env)  # Create a new alignment object for this variant

        # Add the model to the alignment object using the PDB file
        md1 = Model(env, file=f'{PDB_PATH}{gene}.pdb')
        aln.append_model(md1, align_codes=gene, atom_files=f'{PDB_PATH}{gene}.pdb')

        # Add the PIR file to the alignment object
        aln.append(file=f'{PIR_PATH}{variant}.txt', align_codes=variant_raw)

        # Perform the alignment
        aln.align2d()

        # Write the alignment to a .ali file in PIR format
        aln.write(file=ali_file_path, alignment_format='PIR')

        print(f"Generated alignment file for {variant}")  # Notify the user of the generated file
    else:
        # Notify the user if the file already exists
        print(f"Alignment file for {variant} already exists. Skipping.")

    return row  # Return the processed row



In [17]:
# Display the total number of sequences and existing .ali files
print(f"Number of sequences in CSV: {len(variant_df)}")  # Print the total number of sequences in the CSV
print(f"Number of .ali files: {len([name for name in os.listdir(ALI_PATH) if name.endswith('.ali')])}")  # Count and print the number of .ali files in the directory
print(f"Number of missing files: {len(variant_df) - len([name for name in os.listdir(ALI_PATH)])}")  # Calculate and print the number of missing files


Number of sequences in CSV: 384
Number of .ali files: 384
Number of missing files: 0


In [18]:
# Apply the process_variant function to each row in the DataFrame using parallel processing
variant_df.parallel_apply(process_variant, axis=1)

# Display the updated number of sequences and .ali files after processing
print(f"Number of sequences in CSV: {len(variant_df)}")  # Print the total number of sequences in the CSV again
print(f"Number of .ali files: {len([name for name in os.listdir(ALI_PATH) if name.endswith('.ali')])}")  # Count and print the number of .ali files after processing

Alignment file for pncA_His43Pro already exists. Skipping.Alignment file for atpE_Ala63Pro already exists. Skipping.Alignment file for pncA_Pro54Arg already exists. Skipping.Alignment file for rpoB_Ser450Cys already exists. Skipping.Alignment file for rpoB_Gln429His already exists. Skipping.Alignment file for pncA_Leu172Pro already exists. Skipping.Alignment file for pncA_Val180Ala already exists. Skipping.Alignment file for pncA_Asp8Gly already exists. Skipping.Alignment file for ethA_Asn379Asp already exists. Skipping.Alignment file for pncA_Ala102Pro already exists. Skipping.Alignment file for pncA_Thr76Ile already exists. Skipping.Alignment file for rpoB_Leu449Met already exists. Skipping.











Alignment file for pncA_Pro54Gln already exists. Skipping.Alignment file for atpE_Asp28Ala already exists. Skipping.Alignment file for pncA_Leu182Ser already exists. Skipping.Alignment file for rpoB_Ser450Gln already exists. Skipping.Alignment file for pncA_His51Arg already exists. Ski